# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [76]:
stocks_list = pd.read_csv('sp_500_stocks.csv')
stocks_list["Ticker"]
stocks_list = stocks_list[~stocks_list['Ticker'].str.contains("DISCA|HFC|VIAC|WLTW")]

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
IEX_CLOUD_API_TOKEN = 'pk_b6766281be9b4f469c9bbc9b7ddd0251'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [ ]:
symbol = 'DISCA'
api_url = f"https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
data['latestPrice']

134.76

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [ ]:
data

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [36]:
my_stock = {"Ticker": [], "Stock Price": [], "Market Capitalization": [], "Number of Shares to Buy": []}
for tick in stocks_list["Ticker"][:30]:
    api_url = f"https://cloud.iexapis.com/stable/stock/{tick}/quote?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    my_stock["Ticker"].append(tick)
    my_stock["Stock Price"].append(data['latestPrice'])
    my_stock["Market Capitalization"].append(data['marketCap'])
    my_stock["Number of Shares to Buy"].append("N/A")

df = pd.DataFrame(my_stock)



## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [38]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [77]:
final_df = {"Ticker": [], "Stock Price": [], "Market Capitalization": [], "Number of Shares to Buy": []}

symbol_groups = list(chunks(stocks_list['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

for symbol_string in symbol_strings:
    batch_api_call = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call).json()

    for symbol in symbol_string.split(','):
        final_df["Ticker"].append(symbol)
        final_df["Stock Price"].append(data[symbol]['quote']['latestPrice'])
        final_df["Market Capitalization"].append(data[symbol]['quote']['marketCap'])
        final_df["Number of Shares to Buy"].append('N/A')

stocks_df = pd.DataFrame(final_df)
        
        

In [78]:
stocks_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,156.92,4.645962e+10,N/A
1,AAL,17.02,1.106131e+10,N/A
2,AAP,151.03,9.079658e+09,N/A
3,AAPL,134.76,2.134088e+12,N/A
4,ABBV,153.60,2.716386e+11,N/A
...,...,...,...,...
496,YUM,130.22,3.668139e+10,N/A
497,ZBH,123.38,2.589159e+10,N/A
498,ZBRA,293.70,1.516367e+10,N/A
499,ZION,51.39,7.688856e+09,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [80]:
portfolio_size = input("Enter the number of shares you want to buy: ")
val = float(portfolio_size)

In [81]:
position_size = val/len(stocks_df.index)
no_of_apple_shares = position_size/134.6
print(no_of_apple_shares)

14.829182645110967


In [83]:
for i in range(0, len(stocks_df.index)):
    stocks_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / stocks_df['Stock Price'][i])

stocks_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,156.92,4.645962e+10,12
1,AAL,17.02,1.106131e+10,117
2,AAP,151.03,9.079658e+09,13
3,AAPL,134.76,2.134088e+12,14
4,ABBV,153.60,2.716386e+11,12
...,...,...,...,...
496,YUM,130.22,3.668139e+10,15
497,ZBH,123.38,2.589159e+10,16
498,ZBRA,293.70,1.516367e+10,6
499,ZION,51.39,7.688856e+09,38


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: